<font face="'vazirmatn', 'Vazir', 'B Nazanin', 'XB Zar'" size=4><div dir=rtl>
<h3><center>تمرین دوم درس پردازش زبان‌های طبیعی</center></h3>
<h4><center>چالش درست‌کردن هه کسره (گروه چهار)</center></h4>
<table width='100%' style="border: none;">
    <tr style="border: none; text-align: center;">
        <td style="border: none;"><h5>علیرضا بلال</h5></td>
        <td style="border: none;"><h5>زهرا رجالی</h5></td>
        <td style="border: none;"><h5>جواد راضی</h5></td>
    </tr>
</table>
<h5 style="font-size: 16px;"><center> ترم ۴۰۱۲ </center></h5>
<br/>
<hr/>
<br/>

<font face="'vazirmatn', 'Vazir', 'B Nazanin', 'XB Zar'" size=4><div dir=rtl>
<h3>
    خلاصه کارهای صورت‌گرفته
</h3>
<span>
    در این تمرین، ماژولی را توسعه داده‌ایم که با دریافت یک متن فارسی به عنوان ورودی، ایرادات موسوم به «هکسره» آن را یافته، رفع کرده و متنی بدون خطاهای هکسره را برمی‌گرداند. 
</span>
<br/>
<span>
    {بیان ایده برای نحوه یافتن غلط هکسره}
</span>
<span>
    {ابزارها و پکیج‌های استفاده شده}
</span>
<span>
    {نتایج دستیابی‌شده در تمرین}
</span>


<font face="'vazirmatn', 'Vazir', 'B Nazanin', 'XB Zar'" size=4><div dir=rtl>
<h3>
    خطای هه کسره
</h3>
<div>
    توضیحات کلی در مورد خطای هه کسره و قواعد تشخیصش 
</div>

<font face="'vazirmatn', 'Vazir', 'B Nazanin', 'XB Zar'" size=4><div dir=rtl>
<h3>
    نصب پکیج‌ها و ابزارهای مورد نیاز
</h3>
<div>
    توضیحات
</div>

In [17]:
try:
    import hazm
except:
    %pip install hazm
    
try: 
    import wapiti
except: 
    %pip install wapiti

  Using cached wapiti-0.1.tar.gz (38 kB)
  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'error'
Note: you may need to restart the kernel to use updated packages.


  error: subprocess-exited-with-error
  
  python setup.py egg_info did not run successfully.
  exit code: 1
  
  [6 lines of output]
  Traceback (most recent call last):
    File "<string>", line 2, in <module>
    File "<pip-setuptools-caller>", line 34, in <module>
    File "C:\Users\Javad\AppData\Local\Temp\pip-install-7p3ct5uk\wapiti_e5ac4b9c34ce45a5913aef68248e3e2e\setup.py", line 28, in <module>
      raise NotImplementedError("wapiti Python 3 support en route to your location")
  NotImplementedError: wapiti Python 3 support en route to your location
  [end of output]
  
  note: This error originates from a subprocess, and is likely not a problem with pip.
error: metadata-generation-failed

Encountered error while generating package metadata.

See above for output.

note: This is an issue with the package mentioned above, not pip.
hint: See above for details.


In [18]:
from hazm import Normalizer, WordTokenizer, POSTagger


<font face="'vazirmatn', 'Vazir', 'B Nazanin', 'XB Zar'" size=4><div dir=rtl>
<h3>
    هم‌خوان‌سازی کد با کتاب‌خانه parsi-io
</h3>
<div>
    توضیحات
</div>

<font face="'vazirmatn', 'Vazir', 'B Nazanin', 'XB Zar'" size=4><div dir=rtl>
<h3>
    یافتن و تصحیح ایرادات هه کسره
</h3>
<div>
</div>

<h4>
    بیان روش کلی
</h4>
<div>
    توضیحات
</div>

<font face="'vazirmatn', 'Vazir', 'B Nazanin', 'XB Zar'" size=4><div dir=rtl>
<h3>
    پیاده‌سازی کلاس HeKasraExtractor
</h3>
<div>
    توضیحات
</div>

In [19]:
class HeKasraCorrection: 
    def __init__(processed_text):
        self.processed_text = processed_text
        self.corrections = {
            'correct': processed_text['raw_text'],
        }
    
    def correct_token(self, invalid_token, corrected_token, str_index):
        if invalid_token in self.corrections:
            return
        self.corrections[invalid_token] = [str_index, len(invalid_token)]
        corrected_form = self.corrections['correct'][:str_index] + corrected_token + self.corrections['correct'][str_index+len(invalid_token):]
        self.corrections['correct'] = corrected_form
        return self.corrections
    
    def finalize(self):
        return {
            **self.processed_text,
            'corrections': self.corrections,
        }
    
        

In [20]:

class HeKasraExtractor:
    def __init__(self):
        pass
    
    def preprocess(self, text):
        normalizer = Normalizer()
        normalized_text = normalizer.normalize(text)
        tokenizer = WordTokenizer()
        tokens = tokenizer.tokenize(normalized_text)
        tagger = POSTagger(model="model/postagger.model")
        tagged_tokens = tagger.tag(tokens)
        return {
            'raw_text': text,
            'normalized_text': normalized_text,
            'tokens': tokens,
            'pos_tags': tagged_tokens
        }

    # Define rules to identify "He Kasrah" grammar error
    def find_he_kasra_errors(self, processed_text):
        he_kasra_corrections = HeKasraCorrection(processed_text)
        
        # Add your rules based on POS tags and word patterns
        # Example: if tag == 'N' and word.endswith('ه'): return True
        return he_kasra_corrections.finalize()


    def run(self, input_sentence):
        processed = self.preprocess(input_sentence)
        he_kasra_corrections = self.find_he_kasra_errors(processed)
        return he_kasra_corrections['corrections']


In [21]:
hkasra_extractor = HeKasraExtractor()
input_samples = ['کوروشه کبیر', 'حال من خوب است.', 'حاله من خوبه', 'من اگه کتابه تو رو داشتم', 'این دختره دیوانه', 'گل زیبا', 'گله زیبایی را تقدیم کردم']

for sample in input_samples:
    print(hkasra_extractor.preprocess(sample))

ModuleNotFoundError: No module named 'wapiti'

<font face="'vazirmatn', 'Vazir', 'B Nazanin', 'XB Zar'" size=4><div dir=rtl>
<h3>
    ارزیابی ماژول
</h3>
<div>
    توضیحات
</div>

<font face="'vazirmatn', 'Vazir', 'B Nazanin', 'XB Zar'" size=4><div dir=rtl>
<h4>
    خطای ه اضافه بجای نقش‌نمای اضافه
</h4>
<div>
    توضیحات
</div>

In [ ]:
pass

<font face="'vazirmatn', 'Vazir', 'B Nazanin', 'XB Zar'" size=4><div dir=rtl>
<h4>
    خطای عدم جای‌گذاری هکسره
</h4>
<div>
    توضیحات
</div>

In [ ]:
pass

<font face="'vazirmatn', 'Vazir', 'B Nazanin', 'XB Zar'" size=4><div dir=rtl>
<h3>
    تست عملکرد ماژول با تست‌های اتوماتیک
</h3>
<div>
    توضیحات
</div>

In [ ]:
class HeHasraTests:
    pass